# Map Agent with Custom Map Servers

This notebook demonstrates the Map Agent that uses OpenRouteService and OpenStreetMap servers as tools for a Gemini LLM.

## Features:
- **OSM Geocoding**: Convert place names to coordinates
- **OSM Reverse Geocoding**: Convert coordinates to addresses
- **OSM POI Search**: Find points of interest
- **ORS Routing**: Get driving directions
- **ORS Distance**: Calculate distances between points
- **ORS Nearby**: Find nearby points of interest

## Setup and Imports

In [1]:
import os
import sys
from dotenv import load_dotenv
import importlib

# Load environment variables
load_dotenv()

# Set environment variables for Gemini and OSM
os.environ['GEMINI_MODEL'] = os.getenv('GEMINI_MODEL', 'gemini-2.0-flash')
os.environ['OSM_COUNTRY_CODES'] = os.getenv('OSM_COUNTRY_CODES', 'lb')

# Force reload of modules to avoid caching issues
if 'part2_implementation.gemini_provider' in sys.modules:
    del sys.modules['part2_implementation.gemini_provider']
if 'part2_implementation.agent_sdk_app' in sys.modules:
    del sys.modules['part2_implementation.agent_sdk_app']
if 'part2_implementation.servers.osm_server' in sys.modules:
    del sys.modules['part2_implementation.servers.osm_server']
if 'part2_implementation.servers.ors_server' in sys.modules:
    del sys.modules['part2_implementation.servers.ors_server']

# Add part2_implementation to path
sys.path.insert(0, os.path.join(os.getcwd(), 'part2_implementation'))

# Import tools and agent
from part2_implementation.gemini_provider import run_with_tools
from part2_implementation.agent_sdk_app import TOOLS, agent

print("[OK] Environment setup complete")
print(f"[OK] Loaded {len(TOOLS)} tools")
print(f"[OK] Using model: {os.environ['GEMINI_MODEL']}")

[OK] Environment setup complete
[OK] Loaded 6 tools
[OK] Using model: gemini-2.0-flash


C:\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Test Cases

Below are individual test cases for each tool functionality.

### Test 1: OSM Geocoding
Find the coordinates of Bhamdoun, the famous mountain resort town

In [4]:
response = await run_with_tools(
    "Find the coordinates of Bhamdoun, Lebanon - the famous mountain resort town",
    TOOLS,
    agent
)
print(response)


User: Find the coordinates of Bhamdoun, Lebanon - the famous mountain resort town

The coordinates of Bhamdoun, Lebanon are: latitude 33.8030684, longitude 35.6594457.



### Test 2: OSM Reverse Geocoding
Find what's located at specific coordinates near Bhamdoun

In [5]:
response = await run_with_tools(
    "What place is located at coordinates 33.8080, 35.6450? I think it's near a famous mountain resort",
    TOOLS,
    agent
)
print(response)


User: What place is located at coordinates 33.8080, 35.6450? I think it's near a famous mountain resort

The coordinates 33.8080, 35.6450 are located in بحمدون المحطة, لبنان.



### Test 3: OSM POI Search
Find cafes and restaurants in the mountain town of Bhamdoun

In [6]:
response = await run_with_tools(
    "Find cafes and restaurants in Bhamdoun, the mountain resort town",
    TOOLS,
    agent
)
print(response)


User: Find cafes and restaurants in Bhamdoun, the mountain resort town

OK. I found 6 cafes and 10 restaurants in Bhamdoun. Here are some of the results:

Cafes: Nazem cafe, corniche view sawfar, chez bro cafe, green hut coffee shop.
Restaurants: Akram, le telegraphe de belle vue, kahwet al day3a, Aashta wa Easal, Eal Ousoul, Jalset Bhamdoûn, Abo-Gabi, carole bakery, L'Allouette, Adel bakery.



### Test 4: ORS Routing
Get driving route from Beirut to Bhamdoun mountain resort

In [7]:
response = await run_with_tools(
    "Find a scenic driving route from Beirut to Bhamdoun mountain resort and tell me about it",
    TOOLS,
    agent
)
print(response)


User: Find a scenic driving route from Beirut to Bhamdoun mountain resort and tell me about it

The scenic driving route from Beirut to Bhamdoun is approximately 22.28 km and takes about 17 minutes. The route starts in Beirut, heading east on شارع عبد الحميد الزهراوي, then turning right onto شارع إبراهيم الأحدب. You'll then turn sharp right onto جادة إلياس سركيس, keeping left. Continue onto Rue Seif Eddine El-Khatib, then turn left onto شارع حبيب باشا السعد. Continue straight, then keep left before turning slight right onto the A30. Keep right multiple times before turning right again, and you will arrive at your destination on the left in Bhamdoun. Enjoy the beautiful mountain views along the way!



### Test 5: ORS Distance
Calculate distance from Beirut to the mountain escape of Bhamdoun

In [8]:
response = await run_with_tools(
    "What's the driving distance from Beirut to Bhamdoun mountain resort? How long would it take?",
    TOOLS,
    agent
)
print(response)


User: What's the driving distance from Beirut to Bhamdoun mountain resort? How long would it take?

Sorry, I can't find Bhamdoun mountain resort. Can you provide coordinates for that location?



### Test 6: Custom Query
Try your own query here - explore Lebanon's beautiful locations!

In [9]:
# Enter your custom query here
custom_query = "Find hotels and chalets near Bhamdoun for a weekend getaway"

response = await run_with_tools(
    custom_query,
    TOOLS,
    agent
)
print(response)


User: Find hotels and chalets near Bhamdoun for a weekend getaway

I found 4 hotels in Bhamdoun: Al Sheikh, elite hotel 1, M30, and Aley Suites. I didn't find any chalets in Bhamdoun.


## Gradio Web Interface

Launch a web interface to interact with the Map Agent

In [10]:
import gradio as gr
import asyncio

# Define the chat function
async def chat_with_map_agent(message, history):
    """
    Process user message and return agent response.
    
    Args:
        message: User's message
        history: Chat history (not used but required by Gradio)
    
    Returns:
        Agent's response
    """
    try:
        response = await run_with_tools(message, TOOLS, agent)
        return response
    except Exception as e:
        return f"Error: {str(e)}"

# Create Gradio interface
with gr.Blocks(title="Map Agent") as demo:
    gr.Markdown("""
    # Map Agent Assistant - Explore Lebanon
    
    Ask me about beautiful places in Lebanon:
    - Finding mountain resorts like Bhamdoun
    - Getting directions to scenic locations
    - Calculating distances between cities
    - Searching for hotels, cafes, and restaurants
    
    **Example queries:**
    - "Find me a mountain resort hotel in Bhamdoun"
    - "What's the distance from Beirut to Bhamdoun?"
    - "Get me a scenic route from Beirut to Bhamdoun"
    - "Find cafes and restaurants in Bhamdoun"
    - "What historic hotels are near the mountains?"
    """)
    
    chatbot = gr.Chatbot(
        label="Chat with Map Agent",
        height=400
    )
    
    with gr.Row():
        msg = gr.Textbox(
            label="Your message",
            placeholder="Ask about places in Lebanon...",
            scale=4
        )
        send_btn = gr.Button("Send", scale=1)
    
    clear = gr.Button("Clear Chat")
    
    async def respond(message, chat_history):
        """Handle user message and update chat."""
        if not message.strip():
            return "", chat_history
        
        bot_response = await chat_with_map_agent(message, chat_history)
        chat_history.append((message, bot_response))
        return "", chat_history
    
    # Event handlers
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    send_btn.click(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

# Launch the interface
print("\nLaunching Gradio interface...")
print("The interface will open in your browser.")
print("You can also access it at the URL shown below.\n")

demo.launch(share=False)

C:\Users\user\AppData\Local\Temp\ipykernel_11488\1830170957.py:41: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(



Launching Gradio interface...
The interface will open in your browser.
You can also access it at the URL shown below.

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.



User: what are some nice cafes in zeitouna bay beirut



## Additional Testing

Test individual tools directly (without LLM)

In [ ]:
# Test direct tool calls
from part2_implementation.servers.osm_server import OSMServer
from part2_implementation.servers.ors_server import ORSServer

osm = OSMServer()
ors = ORSServer()

# Test geocoding (now synchronous)
print("Testing geocoding for Bhamdoun...")
result = osm.geocode("Bhamdoun, Lebanon")
print(result)

# Test reverse geocoding (now synchronous)
print("\nTesting reverse geocoding near Bhamdoun...")
result = osm.reverse(33.8080, 35.6450)
print(result)

## Summary

This notebook demonstrates:
1. Integration of OpenStreetMap and OpenRouteService as tools
2. Using Gemini LLM to orchestrate tool calls
3. Testing various map-related queries
4. A user-friendly Gradio web interface

You can modify the queries and test different scenarios. Make sure you have:
- Valid API keys in your .env file
- Installed all required dependencies: `pip install -r requirements.txt`